In [48]:
import os

import numpy as np
import scipy.fft as fft
from scipy.special import erf

import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

from tqdm.notebook import tqdm

%matplotlib inline

In [49]:
kwfft = {'norm': 'forward', 'workers': 5}

## Analysis simulation

In [50]:
# Path to data

pathFile = "./output_simu_0.npz"
pathFigures = './figures/'

pathEnergies = './energies0.txt'
pathCorrelations = './correlations0.txt'

pathSave_tavg = './data_tavg'

savePlots = True
maxResol_plots = 1024

In [ ]:
# Read the flow fields 

file_simu = np.load(pathFile)

print(file_simu.files)

x, y, t = file_simu['x'], file_simu['y'], file_simu['t']
k_xx, k_yy = file_simu['kx'], file_simu['ky']

print(f"{t.size} data points on [{t.min()} - {t.max()}]")

M_hat_save = file_simu['M_hat']

In [ ]:
# Plot the evolution of dt

fig, (ax, axb) = plt.subplots(figsize=(6,3), ncols=2, tight_layout=True)
fig.suptitle(f'{t.size} flowfields in [{t.min()} - {t.max()}]')

ax.plot(t[:-1], np.diff(t))
ax.set_xlabel('Time $t$')
ax.set_ylabel('Increments $dt$')
ax.set_ybound([0, ax.get_ybound()[1]*1.1])

axb.plot(t)
axb.set_xlabel('Index')
axb.set_ylabel('Time $t$')

plt.show()

In [6]:
# Pad M_hat with zeros

resol = k_xx.shape[1]
k_1d = k_xx[0]

if resol != M_hat_save.shape[-1]:

    mask_aliasing_1d = ~(3 * np.abs(k_1d) < resol)
    assert np.sum(~mask_aliasing_1d) == M_hat_save.shape[-1]

    M_hat_save_extended = np.zeros((M_hat_save.shape[0], resol, resol), dtype='complex')

    j_dealiased = 0
    for j in range(resol):

        # If should be zero, skip  
        if mask_aliasing_1d[j]:
            continue

        # Else, copy the corresponding column 
        M_hat_save_extended[:,~mask_aliasing_1d,j] = M_hat_save[:,:,j_dealiased]
        j_dealiased += 1 

    M_hat_save = M_hat_save_extended

In [7]:
# Compute fields and derivatives

deriv_x = 1j * k_xx
deriv_y = 1j * k_yy
laplacian = -(k_xx**2 + k_yy**2)

psi_rms = file_simu['psi_rms']

if 'psi_hat' in file_simu.files:
    psi_hat = file_simu['psi_hat']
    psi_hat /= np.sqrt(np.sum(np.abs(psi_hat)**2)) 
    psi = fft.ifft2(psi_hat, **kwfft).real
elif 'psi' in file_simu.files:
    psi = file_simu['psi']
    psi /= np.sqrt(np.mean(np.abs(psi)**2)) 
    psi_hat = fft.fft2(psi, **kwfft)
else:
    raise AssertionError("'psi' or 'psi_hat' should be given in the file")

dx_psi = fft.ifft2(deriv_x * psi_hat, **kwfft).real
dy_psi = fft.ifft2(deriv_y * psi_hat, **kwfft).real
lapl_psi = fft.ifft2(laplacian * psi_hat, **kwfft).real

In [8]:
# Compute M

M_save = fft.ifft2(M_hat_save, **kwfft)

In [9]:
# For max resolution plots
skipSome = resol // maxResol_plots if resol > maxResol_plots else 1

In [ ]:
# Plot (U, V) fields and |M|

indices = [100, 150]

cmap = plt.get_cmap('RdBu_r')

mag = 2
norm = plt.Normalize(vmin=-mag, vmax=mag)
norm2 = plt.Normalize(vmin=1-mag/2, vmax=1+mag/2)

fig, axes = plt.subplots(figsize=(14,4*len(indices)), ncols=3, nrows=len(indices), tight_layout=True, sharex=True, sharey=True)

for i, index in enumerate(indices):

    if len(indices) == 1:
        ax = axes
    else:
        ax = axes[i]

    p1 = ax[0].pcolormesh(x[::skipSome,::skipSome], y[::skipSome,::skipSome], np.real(M_save[index])[::skipSome,::skipSome], cmap=cmap, norm=norm)
    p2 = ax[1].pcolormesh(x[::skipSome,::skipSome], y[::skipSome,::skipSome], np.imag(M_save[index])[::skipSome,::skipSome], cmap=cmap, norm=norm)
    p3 = ax[2].pcolormesh(x[::skipSome,::skipSome], y[::skipSome,::skipSome], np.abs(M_save[index])[::skipSome,::skipSome], cmap=cmap, norm=norm2)

    fig.colorbar(p1, ax=ax[0])
    fig.colorbar(p2, ax=ax[1])
    fig.colorbar(p3, ax=ax[2])

    ax[0].set_title('Horizontal velocity $U$' + f'  $\\left( t = {t[index]:.1f} \\right)$')
    ax[1].set_title('Vertical velocity $V$' + f'  $\\left( t = {t[index]:.1f} \\right)$')
    ax[2].set_title('Velocity magnitude $|M|$' + f'  $\\left( t = {t[index]:.1f} \\right)$')

for axij in axes.ravel():
    axij.set_xlabel('$x$')
    axij.set_ylabel('$y$')
    axij.set_aspect(1)

plt.show()

In [11]:
# Read the energies

energies = pd.read_csv(pathEnergies, delim_whitespace=True)

time_E = energies['time']
kineticEnergy = energies['kineticEnergy']
advectionEnergy = energies['advectionEnergy']
dispersionEnergy = energies['dispersionEnergy']
refractionEnergy = energies['refractionEnergy']

In [ ]:
# Plot the evolution of the energies (Danioux et al.)

mask_time = (time_E < np.inf)
DaniouxEnergy = advectionEnergy + dispersionEnergy + refractionEnergy

fig, (ax, ax2) = plt.subplots(ncols=2, figsize=(9,4), tight_layout=True)

ax.plot(time_E[mask_time], DaniouxEnergy[mask_time], 'k-', label="Total")
ax.plot(time_E[mask_time], advectionEnergy[mask_time], 'k--', label="Advection")
ax.plot(time_E[mask_time], dispersionEnergy[mask_time], 'k:', label="Dispersion")
ax.plot(time_E[mask_time], refractionEnergy[mask_time], 'k-.', label="Refraction")

ax2.plot(time_E[mask_time], np.abs(kineticEnergy[mask_time] - kineticEnergy[0]), 'k--', label='Kinetic energy')
ax2.plot(time_E[mask_time], np.abs(DaniouxEnergy[mask_time] - DaniouxEnergy[0]), 'k-', label="Danioux invariant")

ax.set_title('Energy balance (Danioux)')
ax.legend(loc='upper left')
ax2.set_title('Absolute error on invariants')
ax2.legend(loc='lower left')

ax2.set_yscale('log')

for axi in (ax, ax2):
    axi.set_xlabel('Time')
    axi.set_ylabel('Energy')

if savePlots: plt.savefig(pathFigures + 'invariants.svg')
plt.show()

In [13]:
# Read correlations

correlations = pd.read_csv(pathCorrelations, delim_whitespace=True)

time_C = correlations['time']
corrKE_vortPsi = correlations['KE_vortPsi']
corrKE_KEpsi = correlations['KE_KEpsi']
corrKE_potPsi = correlations['KE_potPsi']
corrPE_vortPsi = correlations['PE_vortPsi']
corrPE_KEpsi = correlations['PE_KEpsi']
corrPE_potPsi = correlations['PE_potPsi']

In [ ]:
# Plot the evolution of the correlations with |M|² over time

fig, (ax, ax2) = plt.subplots(ncols=2, figsize=(10,5), tight_layout=True)

ax.plot(time_C, corrKE_vortPsi, 'k-', label='$\\Delta \\Psi$')
ax.plot(time_C, corrKE_KEpsi, 'k--', label='$|\\nabla \\Psi|^2$')
ax.plot(time_C, corrKE_potPsi, 'k:', label='$\\frac{1}{2}\\left( \gamma^{-1} \\Delta \\Psi - |\\nabla \\Psi|^2 \\right)$')

ax2.plot(time_C, corrKE_vortPsi**2, 'k-', label='$\\Delta \\Psi$')
ax2.plot(time_C, corrKE_KEpsi**2, 'k--', label='$|\\nabla \\Psi|^2$')
ax2.plot(time_C, corrKE_potPsi**2, 'k:', label='$\\frac{1}{2}\\left(\gamma^{-1} \\Delta \\Psi - |\\nabla \\Psi|^2 \\right)$')

for axi in (ax, ax2):
    axi.set_title("Correlation of wave KE with the background flow")
    axi.legend()
    axi.set_xlabel('Time')

ax.set_ylabel('$C_x(|M|^2, f(\\Psi))$')
ax2.set_ylabel('$C_x(|M|^2, f(\\Psi))^2$')

if savePlots: plt.savefig(pathFigures + 'correlation_KE_wave.svg')
plt.show()

In [ ]:
# Plot the evolution of the correlations with |grad(M)|² over time

fig, (ax, ax2) = plt.subplots(ncols=2, figsize=(10,5), tight_layout=True)

ax.plot(time_C, corrPE_vortPsi, 'k-', label='$\\Delta \\Psi$')
ax.plot(time_C, corrPE_KEpsi, 'k--', label='$|\\nabla \\Psi|^2$')
ax.plot(time_C, corrPE_potPsi, 'k:', label='$\\frac{1}{2}\\left( \gamma^{-1} \\Delta \\Psi - |\\nabla \\Psi|^2 \\right)$')

ax2.plot(time_C, corrPE_vortPsi**2, 'k-', label='$\\Delta \\Psi$')
ax2.plot(time_C, corrPE_KEpsi**2, 'k--', label='$|\\nabla \\Psi|^2$')
ax2.plot(time_C, corrPE_potPsi**2, 'k:', label='$\\frac{1}{2}\\left( \gamma^{-1} \\Delta \\Psi -|\\nabla \\Psi|^2 \\right)$')

for axi in (ax, ax2):
    axi.set_title("Correlation of wave PE with the background flow")
    axi.legend(loc='upper right')
    axi.set_xlabel('Time')

ax.set_ylabel('$C_x(|\\nabla M|^2, f(\\Psi))$')
ax2.set_ylabel('$C_x(|\\nabla M|^2, f(\\Psi))^2$')

if savePlots: plt.savefig(pathFigures + 'correlation_PE_wave.svg')
plt.show()

In [16]:
# Geostrophic fields local

vortPsi_local = lapl_psi
bilaplPsi_local = fft.ifft2(laplacian**2 * psi_hat, **kwfft).real
KEpsi_local = np.abs(dx_psi)**2 + np.abs(dy_psi)**2
potentialPsi_local = 0.5 * (vortPsi_local / psi_rms - KEpsi_local)

In [ ]:
# Plot the background field (vorticity, kinetic energy and effective potential)

cmap1 = plt.get_cmap('viridis')
cmap2 = plt.get_cmap('RdBu_r')
cmap3 = plt.get_cmap('RdBu_r')

mag1 = np.sqrt(np.max(KEpsi_local))
mag2 = np.max(np.abs(vortPsi_local))
mag3 = np.max(np.abs(psi))

fig, (ax0, ax1, ax2) = plt.subplots(ncols=3, figsize=(18,5), tight_layout=True)

pc0 = ax0.pcolormesh(x[::skipSome,::skipSome], y[::skipSome,::skipSome], psi[::skipSome,::skipSome], cmap=cmap3, vmin=-mag3, vmax=mag3)
pc1 = ax1.pcolormesh(x[::skipSome,::skipSome], y[::skipSome,::skipSome], np.sqrt(KEpsi_local[::skipSome,::skipSome]), cmap=cmap1, vmin=0, vmax=mag1)
pc2 = ax2.pcolormesh(x[::skipSome,::skipSome], y[::skipSome,::skipSome], vortPsi_local[::skipSome,::skipSome], cmap=cmap2, vmin=-mag2, vmax=mag2)


fig.colorbar(pc0, ax=ax0, label='$\\chi $')
fig.colorbar(pc1, ax=ax1, label='$|\\nabla \\chi|$')
fig.colorbar(pc2, ax=ax2, label='$\\Delta \\chi $')

for axi in (ax0, ax1, ax2):
    axi.set_aspect(1)
    axi.set_xlabel('x')
    axi.set_ylabel('y')
    axi.contour(x[::skipSome,::skipSome], y[::skipSome,::skipSome], np.sqrt(KEpsi_local[::skipSome,::skipSome]), colors='k', linewidths=0.2)

ax0.set_title('Background stream-function')
ax1.set_title('Background velocity')
ax2.set_title('Background vorticity')

if savePlots: plt.savefig(pathFigures + 'background_fields.png', dpi=100)
plt.show()

## Check statistical cvg

In [18]:
# Functions to compute the moving average

# Compute integrations bounds at each time
def compute_bounds_moving_average(time, integration_time, verbose=False):
    # Find first[k] index such that time[i+k] - time[i] >= integration_length 
    nbPoints_average = np.array([np.argmax((time >= time[i] + integration_time)) - i for i in range(time.size)])

    # Only keep points such that time[i] + T <= t_max
    points_valid_for_moving_average = np.argmax((nbPoints_average < 0))
    if verbose: print(f"{points_valid_for_moving_average} points valid for moving average")

    return nbPoints_average, points_valid_for_moving_average

# Compute the moving average
def compute_moving_average(time, signal, nbPoints_average, points_valid_for_moving_average):
    cumulative_integral = np.cumsum(signal * np.gradient(time))

    compute_moving_average = [
        (cumulative_integral[i + nbPoints_average[i]] - cumulative_integral[i]) / (time[i + nbPoints_average[i]] - time[i])
        for i in range(points_valid_for_moving_average)
    ]

    return np.array(compute_moving_average)

def compute_mean(t_start, t_end, time, signal):
    i_start = np.argmin(np.abs(time - t_start))
    i_end = np.argmin(np.abs(time - t_end))
    slice_time = np.slice(i_start, i_end+1)

    return np.sum(signal[slice_time] * np.gradient(time)[slice_time]) / (time[i_end] - time[i_start])


In [ ]:
# Convergence energies Danioux

integration_time_E = 5
nbPoints_average_E, nbPoints_valid_E = compute_bounds_moving_average(time_E, integration_time_E, verbose=False)

dispersionEnergy_mvAvg = compute_moving_average(time_E, dispersionEnergy, nbPoints_average_E, nbPoints_valid_E)
refractionEnergy_mvAvg = compute_moving_average(time_E, refractionEnergy, nbPoints_average_E, nbPoints_valid_E)
advectionEnergy_mvAvg = compute_moving_average(time_E, advectionEnergy, nbPoints_average_E, nbPoints_valid_E)

fig, ax = plt.subplots(figsize=(4,4), tight_layout=True)

ax.axhline(0, c='k', ls=':')
ax.plot(time_E[:nbPoints_valid_E], dispersionEnergy_mvAvg, label="Dispersion")
ax.plot(time_E[:nbPoints_valid_E], refractionEnergy_mvAvg, label="Refraction")
ax.plot(time_E[:nbPoints_valid_E], advectionEnergy_mvAvg, label="Advection")

ax.legend()
ax.set_xlabel("Time $t$")
ax.set_ylabel("Energy ($\\gamma \\times E$)")
ax.set_title(f"Moving $t$-average of energies ($T = {integration_time_E}$)")

maxlim = np.max(np.abs(ax.get_ybound()))
ax.set_ybound([-maxlim, maxlim])

if savePlots: plt.savefig(pathFigures + f'moving_averages_energies_dt{integration_time_E}.svg')
plt.show()

In [ ]:
# Convergence correlations 

integration_time_C = 5
nbPoints_average_C, nbPoints_valid_C = compute_bounds_moving_average(time_C, integration_time_C, verbose=False)

corrKE_vortPsi_mvAvg = compute_moving_average(time_C, corrKE_vortPsi, nbPoints_average_C, nbPoints_valid_C)
corrKE_KEpsi_mvAvg = compute_moving_average(time_C, corrKE_KEpsi, nbPoints_average_C, nbPoints_valid_C)
corrKE_potPsi_mvAvg = compute_moving_average(time_C, corrKE_potPsi, nbPoints_average_C, nbPoints_valid_C)
corrPE_vortPsi_mvAvg = compute_moving_average(time_C, corrPE_vortPsi, nbPoints_average_C, nbPoints_valid_C)
corrPE_KEpsi_mvAvg = compute_moving_average(time_C, corrPE_KEpsi, nbPoints_average_C, nbPoints_valid_C)
corrPE_potPsi_mvAvg = compute_moving_average(time_C, corrPE_potPsi, nbPoints_average_C, nbPoints_valid_C)

fig, ax = plt.subplots(figsize=(8,4), ncols=2, tight_layout=True)

# Correlations wave KE
ax[0].axhline(0, c='k', ls=':')
ax[0].plot(time_C[:nbPoints_valid_C], corrKE_vortPsi_mvAvg, label="$\\Delta \\chi$")
ax[0].plot(time_C[:nbPoints_valid_C], corrKE_KEpsi_mvAvg, label="$|\\nabla \\chi|^2$")
ax[0].plot(time_C[:nbPoints_valid_C], corrKE_potPsi_mvAvg, label="$\\frac{1}{2}\\left( \gamma^{-1} \\Delta \\chi - |\\nabla \\chi|^2 \\right)$")

ax[0].legend()
ax[0].set_xlabel("Time $t$")
ax[0].set_ylabel("Correlations with $|M|^2$")
ax[0].set_title(f"Moving $t$-average of correlations ($T = {integration_time_C}$)")

# Correlations wave PE
ax[1].axhline(0, c='k', ls=':')
ax[1].plot(time_C[:nbPoints_valid_C], corrPE_vortPsi_mvAvg, label="$\\Delta \\chi$")
ax[1].plot(time_C[:nbPoints_valid_C], corrPE_KEpsi_mvAvg, label="$|\\nabla \\chi|^2$")
ax[1].plot(time_C[:nbPoints_valid_C], corrPE_potPsi_mvAvg, label="$\\frac{1}{2}\\left( \gamma^{-1} \\Delta \\chi - |\\nabla \\chi|^2 \\right)$")

ax[1].legend()
ax[1].set_xlabel("Time $t$")
ax[1].set_ylabel("Correlations with $|\\nabla M|^2$")
ax[1].set_title(f"Moving $t$-average of correlations ($T = {integration_time_C}$)")

for axi in ax:
    maxlim = np.max(np.abs(axi.get_ybound()))
    axi.set_ybound([-maxlim, maxlim])

if savePlots: plt.savefig(pathFigures + f'moving_averages_correlations_dt{integration_time_C}.svg')
plt.show()

## Local hamiltonian

In [ ]:
# Compute it at each time

nbTime = M_hat_save.shape[0]

refractionEnergy = 0.5 * lapl_psi * np.abs(M_save)**2 / psi_rms
print("Refraction done")

dispersionEnergy = - 0.5 * M_save.conj() * fft.ifft2(laplacian * M_hat_save, **kwfft) / psi_rms**2
print("Dispersion done")

dx_M = fft.ifft2(deriv_x * M_hat_save, **kwfft); print("dx_M done")
dy_M = fft.ifft2(deriv_y * M_hat_save, **kwfft); print("dy_M done")

advectionEnergy = -1j * M_save.conj() * (dx_psi * dy_M - dy_psi * dx_M) / psi_rms
print("Advection done")

del dx_M, dy_M

hamiltonian = dispersionEnergy + refractionEnergy + advectionEnergy

# Separate it in kinetic and potential contribution
potentialEnergy = potentialPsi_local * np.abs(M_save)**2
kineticEnergy = hamiltonian - potentialEnergy

In [ ]:
# Plot the four energies at one instant and their sum (dispersion/refraction/advection/hamiltonian)

index = 150
cmap = 'RdBu_r'

meanDispersion = dispersionEnergy[index].real.mean()
meanAdvection = advectionEnergy[index].real.mean()
meanRefraction = refractionEnergy[index].mean()
meanHamilt = refractionEnergy[index].real.mean()

# Use the mean of the extrem values for a colorbar centered on zero
magDispersion = np.mean([np.max(dispersionEnergy[index].real), np.max(-dispersionEnergy[index].real)])
magAdvection = np.mean([np.max(advectionEnergy[index].real), np.max(-advectionEnergy[index].real)])
magRefraction = np.mean([np.max(refractionEnergy[index]), np.max(-refractionEnergy[index])])
magHamilt = np.mean([np.max(hamiltonian[index].real), np.max(-hamiltonian[index]).real])

fig, ax = plt.subplots(ncols=2, nrows=2, figsize=(12,10), tight_layout=True, sharex=True, sharey=True)

p0 = ax[0,0].pcolormesh(x[::skipSome,::skipSome], y[::skipSome,::skipSome], dispersionEnergy[index].real[::skipSome,::skipSome], cmap=cmap, norm=mcolors.Normalize(-magDispersion, +magDispersion))
p1 = ax[0,1].pcolormesh(x[::skipSome,::skipSome], y[::skipSome,::skipSome], refractionEnergy[index][::skipSome,::skipSome], cmap=cmap, norm=mcolors.Normalize(-magRefraction, +magRefraction))
p2 = ax[1,0].pcolormesh(x[::skipSome,::skipSome], y[::skipSome,::skipSome], advectionEnergy[index][::skipSome,::skipSome].real, cmap=cmap, norm=mcolors.Normalize(-magAdvection, +magAdvection))
p3 = ax[1,1].pcolormesh(x[::skipSome,::skipSome], y[::skipSome,::skipSome], hamiltonian[index][::skipSome,::skipSome].real, cmap=cmap, norm=mcolors.Normalize(vmin=-magHamilt, vmax=+magHamilt))

# Add colorbars with spatial mean
fig.colorbar(p0, ax=ax[0,0]).ax.axhline(meanDispersion, c='k')
fig.colorbar(p1, ax=ax[0,1]).ax.axhline(meanRefraction, c='k')
fig.colorbar(p2, ax=ax[1,0]).ax.axhline(meanAdvection, c='k')
fig.colorbar(p3, ax=ax[1,1]).ax.axhline(meanHamilt, c='k')

ax[0,0].set_title("Dispersion energy $-\\frac{1}{2\\gamma^2} M^{*}\\Delta M $" + f"  (t = {t[index]:.2f})")
ax[0,1].set_title("Refraction energy $\\frac{1}{2\\gamma} \\left( \\Delta \\chi \\right) |M|^2$" + f"  (t = {t[index]:.2f})")
ax[1,0].set_title("Advection energy $-\\frac{i}{\\gamma} M^{*}J\\left( \\chi, M \\right)$" + f"  (t = {t[index]:.2f})")
ax[1,1].set_title("Total energy $\\langle x,y | \\hat{H} | M \\rangle$" + f"  (t = {t[index]:.2f})")

ax[0,1].contour(x[::skipSome,::skipSome], y[::skipSome,::skipSome], np.sqrt(KEpsi_local[::skipSome,::skipSome]), colors='k', linewidths=0.5)
ax[1,0].contour(x[::skipSome,::skipSome], y[::skipSome,::skipSome], np.sqrt(KEpsi_local[::skipSome,::skipSome]), colors='k', linewidths=0.5)
ax[1,1].contour(x[::skipSome,::skipSome], y[::skipSome,::skipSome], np.sqrt(KEpsi_local[::skipSome,::skipSome]), colors='k', linewidths=0.5)

for axi in ax.ravel():
    axi.set_xlabel('$x$')
    axi.set_ylabel('$y$')
    axi.set_aspect(1)

if savePlots: plt.savefig(pathFigures + f'local_energies_time_{t[index]:.2f}.png', dpi=200)
plt.show()

In [ ]:
# Plot the Hamiltonian (real, imag, modulus)

index = 150
cmap = 'RdBu_r'

# Use the mean of the extrem values for a colorbar centered on zero
magHamil_Re = np.max(np.abs(hamiltonian[index].real)) /2
magHamil_Im = np.max(np.abs(hamiltonian[index].imag)) /2
magHamil_Abs = np.max(np.abs(hamiltonian[index])) /2

fig, ax = plt.subplots(ncols=3, figsize=(18,5), tight_layout=True, sharex=True, sharey=True)

p0 = ax[0].pcolormesh(x[::skipSome,::skipSome], y[::skipSome,::skipSome], hamiltonian[index].real[::skipSome,::skipSome], cmap=cmap, norm=plt.Normalize(-magHamil_Re, +magHamil_Re))
p1 = ax[1].pcolormesh(x[::skipSome,::skipSome], y[::skipSome,::skipSome], hamiltonian[index].imag[::skipSome,::skipSome], cmap=cmap, norm=plt.Normalize(-magHamil_Im, +magHamil_Im))
p2 = ax[2].pcolormesh(x[::skipSome,::skipSome], y[::skipSome,::skipSome], np.abs(hamiltonian[index][::skipSome,::skipSome]), cmap='viridis', norm=plt.Normalize(0, magHamil_Abs))

ax[0].set_title("Real part Hamiltonian   ($Ro_{\\Psi} \\times (tf_0) = $" + f"{t[index]:.2f})")
ax[1].set_title("Imaginary part Hamiltonian   ($Ro_{\\Psi} \\times (tf_0) = $" + f"{t[index]:.2f})")
ax[2].set_title("Modulus Hamiltonian   ($Ro_{\\Psi} \\times (tf_0) = $" + f"{t[index]:.2f})")

fig.colorbar(p0, ax=ax[0])
fig.colorbar(p1, ax=ax[1])
fig.colorbar(p2, ax=ax[2])

ax[0].contour(x[::skipSome,::skipSome], y[::skipSome,::skipSome], np.sqrt(KEpsi_local[::skipSome,::skipSome]), colors='k', linewidths=0.5)
ax[1].contour(x[::skipSome,::skipSome], y[::skipSome,::skipSome], np.sqrt(KEpsi_local[::skipSome,::skipSome]), colors='k', linewidths=0.5)
ax[2].contour(x[::skipSome,::skipSome], y[::skipSome,::skipSome], np.sqrt(KEpsi_local[::skipSome,::skipSome]), colors='k', linewidths=0.5)

for axi in ax.ravel():
    axi.set_xlabel('$x$')
    axi.set_ylabel('$y$')
    axi.set_aspect(1)

if savePlots: plt.savefig(pathFigures + f'local_hamiltonian_time_{t[index]:.2f}.png', dpi=200)
plt.show()

In [ ]:
# Plot the three energies at one instant and their sum (potential/kinetic/hamiltonian)

index = 150
cmap = 'RdBu_r'

meanKinetic = kineticEnergy[index].real.mean()
meanPotential = potentialEnergy[index].mean()
meanHamilt = refractionEnergy[index].real.mean()

# Use the mean of the extrem values for a colorbar centered on zero
magKinetic = np.mean([np.max(kineticEnergy[index].real), np.max(-kineticEnergy[index].real)])
magPotential = np.mean([np.max(potentialEnergy[index]), np.max(-potentialEnergy[index])])
magHamilt = np.mean([np.max(hamiltonian[index].real), np.max(-hamiltonian[index].real)])

fig, ax = plt.subplots(ncols=3, figsize=(18,5), tight_layout=True, sharex=True, sharey=True)

p0 = ax[0].pcolormesh(x[::skipSome,::skipSome], y[::skipSome,::skipSome], kineticEnergy[index].real[::skipSome,::skipSome], cmap=cmap, norm=plt.Normalize(-magKinetic, +magKinetic))
p1 = ax[1].pcolormesh(x[::skipSome,::skipSome], y[::skipSome,::skipSome], potentialEnergy[index][::skipSome,::skipSome], cmap=cmap, norm=plt.Normalize(-magPotential, +magPotential))
p2 = ax[2].pcolormesh(x[::skipSome,::skipSome], y[::skipSome,::skipSome], hamiltonian[index].real[::skipSome,::skipSome], cmap=cmap, norm=plt.Normalize(-magHamilt, +magHamilt))

# Add colorbars with spatial mean
fig.colorbar(p0, ax=ax[0]).ax.axhline(meanKinetic, c='k')
fig.colorbar(p1, ax=ax[1]).ax.axhline(meanPotential, c='k')
fig.colorbar(p2, ax=ax[2]).ax.axhline(meanHamilt, c='k')

ax[0].set_title("QM-Kinetic energy" + f"  (t = {t[index]})")
ax[1].set_title("QM-Potential energy" + f"  (t = {t[index]})")
ax[2].set_title("Total energy $\\langle x,y | \\hat{H} | M \\rangle$" + f"  (t = {t[index]})")


for axi in ax.ravel():
    axi.set_xlabel('$x$')
    axi.set_ylabel('$y$')
    axi.set_aspect(1)

if savePlots: plt.savefig(pathFigures + f'local_energies_QMkinetic_QMpotential_time_{t[index]}.png', dpi=200)
plt.show()

In [ ]:
# Plot histogram advection/dispersion/refraction

index = 150

histDisp, binsDist = np.histogram(dispersionEnergy[index].real, bins='sqrt', density=True)
histDisp_Im, binsDist_Im = np.histogram(dispersionEnergy[index].imag, bins='sqrt', density=True)
histAdvect, binsAdvect = np.histogram(advectionEnergy[index].real, bins='sqrt', density=True)
histAdvect_Im, binsAdvect_Im = np.histogram(advectionEnergy[index].imag, bins='sqrt', density=True)
histRefrac, binsRefrac = np.histogram(refractionEnergy[index], bins='sqrt', density=True)
histKinetic, binsKinetic = np.histogram(kineticEnergy[index].real, bins='sqrt', density=True)
histKinetic_Im, binsKinetic_Im = np.histogram(kineticEnergy[index].imag, bins='sqrt', density=True)

meanDispersion = dispersionEnergy[index].real.mean()
meanAdvection = advectionEnergy[index].real.mean()
meanRefraction = refractionEnergy[index].mean()
meanKinetic = kineticEnergy[index].real.mean()

colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red']
fig, ax = plt.subplots(figsize=(12,4), ncols=3, tight_layout=True)
fig.suptitle('PDF of the components of the hamiltonian')

# Advection / Dispersion / Refraction formulation
ax[0].plot(0.5 * (binsDist[1:] + binsDist[:-1]), histDisp, c=colors[0], label='Dispersion (Re)')
ax[0].plot(0.5 * (binsDist_Im[1:] + binsDist_Im[:-1]), histDisp_Im, c=colors[0], alpha=0.5, label='Dispersion (Im)')
ax[0].plot(0.5 * (binsAdvect[1:] + binsAdvect[:-1]), histAdvect, c=colors[1], label='Advection (Re)')
ax[0].plot(0.5 * (binsAdvect_Im[1:] + binsAdvect_Im[:-1]), histAdvect_Im, c=colors[1], alpha=0.5, label='Advection (Im)')
ax[1].plot(0.5 * (binsRefrac[1:] + binsRefrac[:-1]), histRefrac, c=colors[2], label='Refraction')

ax[0].axvline(meanDispersion, c=colors[0], alpha=0.5)
ax[0].axvline(meanAdvection, c=colors[1], alpha=0.5)
ax[1].axvline(meanRefraction, c=colors[2], alpha=0.5)

# Kinetic / Potential formulation
ax[2].plot(0.5 * (binsKinetic[1:] + binsKinetic[:-1]), histKinetic, c=colors[3], label='Kinetic (Re)')
ax[2].plot(0.5 * (binsKinetic_Im[1:] + binsKinetic_Im[:-1]), histKinetic_Im, c=colors[3], alpha=0.5, label='Kinetic (Im)')
ax[2].axvline(meanKinetic, c=colors[3], alpha=0.5)

for axi in ax:
    axi.legend(frameon=False)
    axi.set_xlabel('Energy')
    axi.set_ylabel('PDF')
    axi.set_yscale('log')
    axi.axvline(0, c='grey', alpha=0.5)

plt.show()

In [ ]:
# Plot the histogram at different times

indicesHist = [0, 2, 5, 10, 50, 100]

nPlots = len(indicesHist)
ncols = nPlots //2
logY = False

fig, ax = plt.subplots(ncols=ncols, nrows=2, figsize=(4*nPlots//2, 8), tight_layout=True, sharey=True)

bins = np.linspace(-10/psi_rms, 10/psi_rms,100)

for i, index in enumerate(indicesHist):
    axij = ax.ravel()[i]
    axij.hist(hamiltonian[index].real.ravel(), bins=bins, label='Wave energy', density=True)
    axij.hist(0.5 * lapl_psi.ravel() / psi_rms, bins=bins, alpha=0.5, color='k', label='$\\frac{1}{2 \\gamma} \\Delta \\chi$', density=True)
    axij.set_title(f'Time $t={t[index]:.2f}$')
    axij.axvline(0, ls=':', c='k')
    if logY: axij.set_yscale('log')
    
ax[0,0].legend(loc='upper left', frameon=False, fontsize='medium')

savename = "histogram_hamiltonian"
if logY: savename + "_logY"

if savePlots: plt.savefig(pathFigures + savename + '.png', dpi=100)
plt.show()

In [ ]:
# Plot the histogram at different times

indicesHist = [0, 2, 5, 10, 50, 100]
gaussian_fit = False
logPlot = False

bins = np.linspace(-10/psi_rms, 10/psi_rms, 300)
midpoints = 0.5 * (bins[1:] + bins[:-1])

colors = plt.cm.inferno(np.linspace(0,0.8, len(indicesHist)))


fig, ax = plt.subplots(figsize=(8,4), ncols=2, tight_layout=True)

ax[0].hist(0.5 * lapl_psi.ravel() / psi_rms, bins=bins, color='k', density=True, alpha=0.5, label='$\\frac{1}{2\\gamma} \\Delta \\chi$')
ax[1].hist(0.5 * lapl_psi.ravel() / psi_rms, bins=bins, color='k', density=True, alpha=0.5, label='$\\frac{1}{2\\gamma} \\Delta \\chi$')

for index, color in zip(indicesHist, colors):
    histHam, _ = np.histogram(hamiltonian[index].real, bins=bins, density=True)
    histHamRed, _ = np.histogram(hamiltonian[index].real / np.abs(M_save[index])**2, bins=bins, density=True)

    ax[0].plot(midpoints, histHam, color=color, label=f'$t = {t[index]:.1f}$')
    ax[1].plot(midpoints, histHamRed, color=color, label=f'$t = {t[index]:.1f}$')

def gaussian(mean=0, std=1.):
    return (lambda x: 1./np.sqrt(2 * np.pi) / std * np.exp(-0.5 * ((x - mean)/std)**2))

# Plot gaussian for particle energy
if gaussian_fit:
    index = np.max(indicesHist)
    meanGaussian = np.mean(hamiltonian[index].real / np.abs(M_save[index])**2)
    stdGaussian = np.std(hamiltonian[index].real / np.abs(M_save[index])**2)
    print(f"Gaussian std: {stdGaussian}")

    gaussian_fit = gaussian(mean=meanGaussian, std=stdGaussian)
    ax[1].plot(midpoints, gaussian_fit(midpoints), color='k', ls=':')
    #ax[1].plot(midpoints, np.histogram(hamiltonian[index] / np.abs(M_save[index])**2, bins=bins, density=True)[0], color='k', ls=':')

ax[0].legend(loc='upper right', frameon=False, fontsize='medium')

ax[0].set_title('Evolution of the Hamiltonian PDF')
ax[0].set_xlabel('Local hamiltonian $M^* \mathcal{H}(M)$')
ax[0].set_ylabel('PDF')

ax[1].set_title('Evolution of the particle energy PDF')
ax[1].set_xlabel('Local mean particle energy $\\frac{ M^* \mathcal{H}(M)}{|M|^2}$')
ax[1].set_ylabel('PDF')
savename = "histogram_hamiltonian_onePlot"

if logPlot:
    ax[0].set_yscale('log')
    ax[1].set_yscale('log')
    savename += "_log"

if savePlots: plt.savefig(pathFigures + savename + '.png', dpi=200)
plt.show()

## Time averaged

### Compute it from flowfields

In [ ]:
# Parameters averaging

t_start = 50
t_end = np.inf

t_start = max(t_start, t.min())
t_end = min(t_end, t.max())

maskTime = (t_start <= t) & (t <= t_end)
print(f"{np.sum(maskTime)} snapshots to average in [{t_start} - {t_end}]")

In [ ]:
# Compute local quadratic fields

dx_M = fft.ifft2(M_hat_save[maskTime] * deriv_x, **kwfft)
dy_M = fft.ifft2(M_hat_save[maskTime] * deriv_y, **kwfft)

print("Derivatives computed")

PEwave_local = np.abs(dx_M)**2 + np.abs(dy_M)**2
KEwave_local = np.abs(M_save[maskTime])**2

print("KE and PE computed")

KEwave_hat = fft.fft2(KEwave_local, **kwfft)

# There is a prefactor U_w / (fL) that can be anything (as long as smaller than Ro_psi for linearization)
# u_s = i/4 * (M gradM* - M* gradM) + 1/4 * rot(KEwave e_z) 
stokesDrift_local_x = 0.25 * np.imag(2 * M_save[maskTime].conj() * dx_M) + 0.25 * fft.ifft2(deriv_y * KEwave_hat, **kwfft).real
stokesDrift_local_y = 0.25 * np.imag(2 * M_save[maskTime].conj() * dy_M) - 0.25 * fft.ifft2(deriv_x * KEwave_hat, **kwfft).real

del KEwave_hat
print("Stokes drift computed")

# Self advection
selfAdvect = - 2 * np.imag(dx_M * dy_M.conj()) 
print("Advection computed")

# Local hamiltonian
hamiltonian = (  0.5 * lapl_psi * np.abs(M_save[maskTime])**2 / psi_rms                                                        # Refraction
               - 0.5 * np.real(M_save[maskTime].conj() * fft.ifft2(laplacian * M_hat_save[maskTime], **kwfft)) / psi_rms**2    # Dispersion
               + np.real(-1j * M_save[maskTime].conj() * (dx_psi * dy_M - dy_psi * dx_M)) / psi_rms)                           # Avection
print("Local hamiltonian computed")

del dx_M, dy_M

KEpsi_local = np.abs(dx_psi)**2 + np.abs(dy_psi)**2

In [31]:
# Time averaging (trapeze rule for varying dt when restart)

PEwave_tavg = np.trapz(PEwave_local, t[maskTime], axis=0) / (t_end - t_start)
KEwave_tavg = np.trapz(KEwave_local, t[maskTime], axis=0) / (t_end - t_start)

M_hat_tavg = np.trapz(M_hat_save[maskTime], t[maskTime], axis=0) / (t_end - t_start)

stokesX_tavg = np.trapz(stokesDrift_local_x, t[maskTime], axis=0) / (t_end - t_start)
stokesY_tavg = np.trapz(stokesDrift_local_y, t[maskTime], axis=0) / (t_end - t_start)

selfAdvect_tavg = np.trapz(selfAdvect, t[maskTime], axis=0) / (t_end - t_start)
hamiltonian_tavg = np.trapz(hamiltonian, t[maskTime], axis=0) / (t_end - t_start)

kineticQM_tavg = KEwave_tavg * potentialPsi_local
stokesNorm = np.sqrt(np.abs(stokesX_tavg)**2 + np.abs(stokesY_tavg)**2)

In [ ]:
# Save time-averaged data

savename = f'{pathSave_tavg}_{int(t_start)}_{int(t_end)}.npz'

assert not os.path.isfile(savename)
np.savez_compressed(savename, 
                    M_hat = M_hat_tavg, 
                    KEwave = KEwave_tavg, PEwave = PEwave_tavg, 
                    psi_hat = psi_hat, psi_rms = psi_rms,
                    stokesDrift = np.array([stokesX_tavg, stokesY_tavg]),
                    selfAdvect = selfAdvect_tavg,
                    hamiltonian = hamiltonian_tavg,
                    )

### Use integral field

In [ ]:
# Use one the fly integrated quadratic quantities to compute mean

path_to_timeIntegrated = './fields0/'
assert os.path.isdir(path_to_timeIntegrated)

filenames_timeIntegrated = [entry for entry in os.listdir(path_to_timeIntegrated) if entry.startswith("tIntegrated") and os.path.isfile(os.path.join(path_to_timeIntegrated, entry))]

key_sort = (lambda name: int(name[len('tIntegrated_'):-len('.npz')]))
filenames_timeIntegrated = sorted(filenames_timeIntegrated, key=key_sort)
print(filenames_timeIntegrated)

times_Integrated = [np.load(os.path.join(path_to_timeIntegrated,filename))['t_bounds'] for filename in filenames_timeIntegrated]
times_Integrated = np.array(list(zip(*times_Integrated)))
print(times_Integrated)

In [ ]:
# Parameters for time average

t_start = 100
t_end = 200

# Compute the closest point to the desired times
argmin_t_start = np.unravel_index(np.argmin(np.abs(times_Integrated.ravel() - t_start)), times_Integrated.shape)
argmin_t_end = np.unravel_index(np.argmin(np.abs(times_Integrated.ravel() - t_end)), times_Integrated.shape)

t_start = times_Integrated[argmin_t_start]
t_end = times_Integrated[argmin_t_end]

assert t_start < t_end, "No data to compute average, change bounds"
print(f"Times for averaging : {t_start} - {t_end}")

In [ ]:
# Compute time-average

if not (argmin_t_start[0] == 1 and argmin_t_end[0] == 1):
    raise AttributeError("Not in the usual case, do it manually")

# Load int_t0^t_end, int_t0^t_start
file_tIntegrated_end = np.load(os.path.join(path_to_timeIntegrated, filenames_timeIntegrated[argmin_t_end[1]]))
file_tIntegrated_start = np.load(os.path.join(path_to_timeIntegrated, filenames_timeIntegrated[argmin_t_start[1]]))
print(file_tIntegrated_end.files)

# Then compute the difference 
KEwave_tAvg = (file_tIntegrated_end['waveKE'] - file_tIntegrated_start['waveKE']) / (t_end - t_start)
PEwave_tAvg = (file_tIntegrated_end['wavePE'] - file_tIntegrated_start['wavePE']) / (t_end - t_start)
stokesX_tAvg = (file_tIntegrated_end['waveStokes_x'] - file_tIntegrated_start['waveStokes_x']) / (t_end - t_start)
stokesY_tAvg = (file_tIntegrated_end['waveStokes_y'] - file_tIntegrated_start['waveStokes_y']) / (t_end - t_start)

In [ ]:
# Compute time-average manually

# Load int_t0^t_end, int_t0^t_start
file_tIntegrated_end = np.load(os.path.join(path_to_timeIntegrated, filenames_timeIntegrated[argmin_t_end[1]]))
print(file_tIntegrated_end.files)
print(*file_tIntegrated_end['t_bounds'])

# Then compute the difference 
KEwave_tAvg = file_tIntegrated_end['waveKE'] / (t_end - t_start)
PEwave_tAvg = file_tIntegrated_end['wavePE'] / (t_end - t_start)
stokesX_tAvg = file_tIntegrated_end['waveStokes_x'] / (t_end - t_start)
stokesY_tAvg = file_tIntegrated_end['waveStokes_y'] / (t_end - t_start)

In [ ]:
# Save time-averaged fields

savepath_tAvg = f"./data_tavg_from_tIntegration_{int(t_start)}_{int(t_end)}.npz"
assert not os.path.isfile(savepath_tAvg)

np.savez(savepath_tAvg,
         t_start=t_start, t_end=t_end,
         KEwave=KEwave_tAvg,
         PEwave=PEwave_tAvg,
         stokesDrift=np.array([stokesX_tAvg, stokesY_tAvg]),
         x=x,y=y,k_xx=k_xx,k_yy=k_yy,
         psi_hat=psi_hat, psi_rms=psi_rms,
         )

### Study it

In [ ]:
# Choose existing files

indexfile = 0

filenames_tavg = sorted([filename for filename in os.listdir('.') if ('data_tavg' in filename) and ('.npz' in filename)])
print(filenames_tavg)

filename_tavg = filenames_tavg[indexfile]
print(filename_tavg)

In [ ]:
# Get times start and end tAvg

t_start, t_end = filename_tavg.split("_")[-2:]
t_end = ''.join(t_end.split('.')[:-1])

t_start = float(t_start)
t_end = float(t_end)

print(t_start)
print(t_end)

In [ ]:
# Load file

file_tavg = np.load(os.path.join('.', filename_tavg))
print(file_tavg.files)

PEwave_tavg = file_tavg['PEwave']
KEwave_tavg = file_tavg['KEwave']

stokes_tavg = file_tavg['stokesDrift']
stokesX_tavg = stokes_tavg[0]
stokesY_tavg = stokes_tavg[1]

selfAdvect_tavg = file_tavg['selfAdvect']
hamiltonian_tavg = file_tavg['hamiltonian']

kineticQM_tavg = KEwave_tavg * potentialPsi_local
stokesNorm = np.sqrt(np.abs(stokesX_tavg)**2 + np.abs(stokesY_tavg)**2)

#t_start = file_tavg['t_start']
#t_end = file_tavg['t_end']
print(f"{t_start} - {t_end}")

In [ ]:
# Plot KE and PE (contour U0)

norm = 'log' # lin, log

cmap = plt.get_cmap('RdBu_r')

max_KE, min_KE = 10, .1
max_PE, min_PE = np.max(PEwave_tavg), 1e1 #max(PEwave_tavg.mean()-PEwave_tavg.std(),1)

if norm == 'lin':
    normKE = mcolors.Normalize(vmin=0, vmax=max_KE)
    normPE = mcolors.Normalize(vmin=0, vmax=max_PE)
elif norm == 'log':
    normKE = mcolors.LogNorm(vmin=1./max_KE, vmax=max_KE)
    normPE = mcolors.LogNorm(vmin=min_PE, vmax=max_PE)
else:
    raise KeyError('No norm')

fig, ax = plt.subplots(figsize=(12,5), ncols=2, tight_layout=True)

p1 = ax[0].pcolormesh(x[::skipSome,::skipSome], y[::skipSome,::skipSome], KEwave_tavg[::skipSome,::skipSome], cmap=cmap, norm=normKE)
p2 = ax[1].pcolormesh(x[::skipSome,::skipSome], y[::skipSome,::skipSome], PEwave_tavg[::skipSome,::skipSome], cmap=cmap, norm=normPE)

for axi in ax:
    cs = axi.contour(x[::skipSome,::skipSome], y[::skipSome,::skipSome], np.sqrt(KEpsi_local[::skipSome,::skipSome]), colors='k', linewidths=0.5)
    #axi.clabel(cs, inline=True, fontsize=10)

fig.colorbar(p1, ax=ax[0], label="$|M|^2$")
fig.colorbar(p2, ax=ax[1], label="$|\\nabla M|^2$")

ax[0].set_title(f'Kinetic energy $\\langle |M|^2 \\rangle _t$ ')
ax[1].set_title(f'Potential energy $\\langle |\\nabla M|^2\\rangle _t$')

for axi in ax:
    axi.set_xlabel('$x$')
    axi.set_ylabel('$y$')
    axi.set_aspect(1)

if savePlots: plt.savefig(pathFigures + f'KE_PE_timeAvg_{int(t_start)}_{int(t_end)}.png', dpi=200)
plt.show()


In [ ]:
# Plot the Stokes drift and background velocity

from scipy.ndimage import gaussian_filter

nSkip = resol // 30
cmap = 'viridis'

fig, ax = plt.subplots(ncols=2, figsize=(12,5), tight_layout=True)

# Background flow
p2 = ax[1].pcolormesh(x[::skipSome,::skipSome], y[::skipSome,::skipSome], 0.5 * psi_rms * np.sqrt(KEpsi_local[::skipSome,::skipSome]), cmap=cmap, norm=plt.Normalize(vmin=0.))
ax[1].quiver(x[::nSkip,::nSkip], y[::nSkip,::nSkip], 
             -dy_psi[::nSkip,::nSkip], dx_psi[::nSkip,::nSkip],
             angles='xy')

fig.colorbar(p2, ax=ax[1], label="$\\frac{1}{2} \| \\nabla \\Psi \|$")
ax[1].set_title('Geostrophic background flow')

# Stokes drift
p1 = ax[0].pcolormesh(x[::skipSome,::skipSome], y[::skipSome,::skipSome], stokesNorm[::skipSome,::skipSome], cmap=cmap, norm=plt.Normalize(vmin=0., vmax=p2.get_clim()[1]))
ax[0].quiver(x[::nSkip,::nSkip], y[::nSkip,::nSkip], 
          gaussian_filter(stokesX_tavg, sigma=nSkip/2, mode='wrap')[::nSkip,::nSkip], 
          gaussian_filter(stokesY_tavg, sigma=nSkip/2, mode='wrap')[::nSkip,::nSkip],
          angles='xy')

fig.colorbar(p1, ax=ax[0], label="$\| \\langle \\vec u_s\\rangle _t \|$")
ax[0].set_title('Stokes drift')

for axi in ax:
    axi.set_xlabel('$x$')
    axi.set_ylabel('$y$')
    axi.set_aspect(1)

if savePlots: plt.savefig(pathFigures + f'stokesDrift_timeAvg_{int(t_start)}_{int(t_end)}.png', dpi=200)
plt.show()

In [ ]:
# Plot the self advection and its histogram

sigma_smoothed = 5
cmap = "RdBu_r"

fig, ax = plt.subplots(ncols=3, figsize=(18,5), tight_layout=True)

magSelfAdvect = np.max(np.abs(selfAdvect_tavg)) 

# Color plot
p1 = ax[0].pcolormesh(x, y, selfAdvect_tavg, cmap=cmap, norm=plt.Normalize(vmin=-magSelfAdvect, vmax=+magSelfAdvect))
fig.colorbar(p1, ax=ax[0], label="$\\langle i J \\left(M, M^* \\right) \\rangle _t$")
ax[0].contour(x[::skipSome,::skipSome], y[::skipSome,::skipSome], np.sqrt(KEpsi_local[::skipSome,::skipSome]), colors='k', linewidths=0.1)

ax[0].set_title("Self-advection")
ax[0].set_xlabel('$x$')
ax[0].set_ylabel('$y$')
ax[0].set_aspect(1)

# Color plot (smoothed)
selfAdvect_tavg_smoothed = gaussian_filter(selfAdvect_tavg, sigma=sigma_smoothed, mode='wrap')
magSelfAdvect_smoothed = np.max(np.abs(selfAdvect_tavg_smoothed))
p1 = ax[1].pcolormesh(x, y, selfAdvect_tavg_smoothed,
                      cmap=cmap, norm=plt.Normalize(-magSelfAdvect_smoothed, magSelfAdvect_smoothed))
fig.colorbar(p1, ax=ax[1], label="$\\langle i J \\left(M, M^* \\right) \\rangle _t$")
ax[1].contour(x[::skipSome,::skipSome], y[::skipSome,::skipSome], np.sqrt(KEpsi_local[::skipSome,::skipSome]), colors='k', linewidths=0.1)

ax[1].set_title("Self-advection (smoothed)")
ax[1].set_xlabel('$x$')
ax[1].set_ylabel('$y$')
ax[1].set_aspect(1)

# Histogram 
hist, bins = np.histogram(selfAdvect_tavg.ravel(), density=True, bins='auto')
ax[2].plot(0.5 * (bins[1:] + bins[:-1]), hist, label="Raw")
hist_smoothed, _ = np.histogram(selfAdvect_tavg_smoothed.ravel(), density=True, bins=bins)
ax[2].plot(0.5 * (bins[1:] + bins[:-1]), hist_smoothed, label="Smoothed")
ax[2].set_title("Self-advection spatial PDF")
ax[2].set_xlabel("$\\langle i J \\left(M, M^* \\right) \\rangle _t$")
ax[2].set_ylabel("Spatial PDF")
ax[2].set_yscale('log')
ax[2].legend()

if savePlots: plt.savefig(pathFigures + f'selfAdvection_timeAvg_{int(t_start)}_{int(t_end)}.png', dpi=200)
plt.show()

### Study the lack of particules in high potential

In [40]:
# Compute the normalization coefficient C = 1/(2pi)^2 int Heaviside(E0 - V(x)) dx

maxPot = np.max(potentialPsi_local)
minPot = np.min(potentialPsi_local)

def normalization_Heaviside(energy):
    if energy > maxPot:
        return 1
    elif energy < minPot:
        return 0
    else:
        heaviside = np.heaviside(energy - potentialPsi_local, 1.)
        return np.mean(heaviside)

#normalization_Heaviside = np.vectorize(normalization_Heaviside)

In [ ]:
# Plot normalization coefficient

energies_E0 = np.linspace(minPot, maxPot, 1000)
normalization_E0 = np.array([normalization_Heaviside(energy) for energy in energies_E0])

fig, ax = plt.subplots(figsize=(4,4), tight_layout=True)

ax.plot(energies_E0, normalization_E0)
ax.axvline(minPot, c='k', ls=':', label='Limits $V$')
ax.axvline(maxPot, c='k', ls=':')

ax.axvline(np.min(0.5 * lapl_psi / psi_rms), c='b', ls=':', label="Limits $E_0$")
ax.axvline(np.max(0.5 * lapl_psi / psi_rms), c='b', ls=':')

ax.set_xlabel('Energy $E_0$')
ax.set_ylabel('Normalization $C_{E_0}$')
ax.set_title('Normalization coefficient')
ax.legend(loc='upper left')

plt.show()

In [ ]:
# Compute integral function

mu_H_E0, _ = np.histogram(0.5 * lapl_psi / psi_rms, bins=energies_E0, density=True)
integrand_energy = mu_H_E0 / normalization_E0[1:]

integral_energy = np.cumsum(integrand_energy) * (energies_E0[1] - energies_E0[0])
integral_energy = integral_energy[-1] - integral_energy # Back to correct ordering

proba_E0_larger_maxV = np.mean(np.heaviside(0.5 * lapl_psi / psi_rms - maxPot, 1.))
print(proba_E0_larger_maxV)

In [43]:
# Compute the interpolation

phyStat_prediction_nonUnif_E0 = np.interp(potentialPsi_local.ravel(), energies_E0[1:], integral_energy) + proba_E0_larger_maxV
phyStat_prediction_nonUnif_E0 = phyStat_prediction_nonUnif_E0.reshape(potentialPsi_local.shape)

In [44]:
# Compute approximate simplification
# Assume that normalization_E0 = 1 and proba_E0_larger_maxV = 0

integral_energy_simple = np.cumsum(mu_H_E0) * (energies_E0[1] - energies_E0[0])
integral_energy_simple = integral_energy_simple[-1] - integral_energy_simple

phyStat_prediction_nonUnif_E0_simple = np.interp(potentialPsi_local.ravel(), energies_E0[1:], integral_energy_simple)
phyStat_prediction_nonUnif_E0_simple = phyStat_prediction_nonUnif_E0_simple.reshape(potentialPsi_local.shape)

In [45]:
# Compute the gaussian prediction

vorticity_rms = np.sqrt(np.mean(lapl_psi**2))

phyStat_prediction_gaussianE0 = 0.5 * (1 - erf((lapl_psi - psi_rms * KEpsi_local) / (np.sqrt(2) * vorticity_rms)))

In [ ]:
# Compare the three predictions

norm = 'lin' # lin, log

cmap = plt.get_cmap('RdBu_r')

max_KEpred = max(np.max(phyStat_prediction_nonUnif_E0), np.max(phyStat_prediction_nonUnif_E0_simple), np.max(phyStat_prediction_gaussianE0))
min_KEpred = min(np.min(phyStat_prediction_nonUnif_E0), np.min(phyStat_prediction_nonUnif_E0_simple), np.min(phyStat_prediction_gaussianE0))

if norm == 'lin':
    #normKE = mcolors.Normalize(vmin=min_KEpred, vmax=max_KEpred)
    normKE = mcolors.Normalize(0, 2)
elif norm == 'log':
    normKE = mcolors.LogNorm(vmin=min_KEpred, vmax=max_KEpred)
else:
    raise KeyError('No norm')

fig, ax = plt.subplots(figsize=(19,5), ncols=3, tight_layout=True)

p1 = ax[0].pcolormesh(x[::skipSome,::skipSome], y[::skipSome,::skipSome], phyStat_prediction_nonUnif_E0[::skipSome,::skipSome], cmap=cmap, norm=normKE)
p2 = ax[1].pcolormesh(x[::skipSome,::skipSome], y[::skipSome,::skipSome], phyStat_prediction_nonUnif_E0_simple[::skipSome,::skipSome], cmap=cmap, norm=normKE)
p3 = ax[2].pcolormesh(x[::skipSome,::skipSome], y[::skipSome,::skipSome], phyStat_prediction_gaussianE0[::skipSome,::skipSome], cmap=cmap, norm=normKE)

ax[0].contour(x[::skipSome,::skipSome], y[::skipSome,::skipSome], np.sqrt(KEpsi_local[::skipSome,::skipSome]), colors='k', linewidths=0.5)
ax[1].contour(x[::skipSome,::skipSome], y[::skipSome,::skipSome], np.sqrt(KEpsi_local[::skipSome,::skipSome]), colors='k', linewidths=0.5)
ax[2].contour(x[::skipSome,::skipSome], y[::skipSome,::skipSome], np.sqrt(KEpsi_local[::skipSome,::skipSome]), colors='k', linewidths=0.5) 

fig.colorbar(p1, ax=ax[0], label="$|M_{pred,1}|^2$")
fig.colorbar(p2, ax=ax[1], label="$|M_{pred,2}|^2$")
fig.colorbar(p3, ax=ax[2], label="$|M_{pred,3}|^2$")

ax[0].set_title(f'Phy Stat prediction (full)')
ax[1].set_title(f'Phy Stat prediction (simplified)')
ax[2].set_title(f'Phy Stat prediction (gaussian)')

for axi in ax:
    axi.set_xlabel('$x$')
    axi.set_ylabel('$y$')
    axi.set_aspect(1)

plt.show()

In [ ]:
# Plot KE and its prediction in the ergodic case

whichPrediction = "gaussian" # "full", "simple", "gaussian"

cmap = plt.get_cmap('RdBu_r')

norm = 'lin' # lin, log
max_KE, min_KE = 2, 0.

if norm == 'lin':
    normKE = mcolors.Normalize(vmin=0, vmax=max_KE)
elif norm == 'log':
    normKE = mcolors.LogNorm(vmin=1./max_KE, vmax=max_KE)
else:
    raise KeyError('No norm')

normError = plt.Normalize(0, 0.5)

fig, ax = plt.subplots(figsize=(18,5), ncols=3, tight_layout=True)

p1 = ax[0].pcolormesh(x[::skipSome,::skipSome], y[::skipSome,::skipSome], KEwave_tavg[::skipSome,::skipSome], cmap=cmap, norm=normKE)

if whichPrediction == "simple":
    p2 = ax[1].pcolormesh(x[::skipSome,::skipSome], y[::skipSome,::skipSome], 
                          phyStat_prediction_nonUnif_E0_simple[::skipSome,::skipSome], cmap=cmap, norm=normKE)
    p3 = ax[2].pcolormesh(x[::skipSome,::skipSome], y[::skipSome,::skipSome], 
                          np.abs(KEwave_tavg[::skipSome,::skipSome] - phyStat_prediction_nonUnif_E0_simple[::skipSome,::skipSome]), cmap='viridis', norm=normError)
elif whichPrediction == "full":
    p2 = ax[1].pcolormesh(x[::skipSome,::skipSome], y[::skipSome,::skipSome], 
                          phyStat_prediction_nonUnif_E0[::skipSome,::skipSome], cmap=cmap, norm=normKE)
    p3 = ax[2].pcolormesh(x[::skipSome,::skipSome], y[::skipSome,::skipSome], 
                          np.abs(KEwave_tavg[::skipSome,::skipSome] - phyStat_prediction_nonUnif_E0[::skipSome,::skipSome]), cmap='viridis', norm=normError)
elif whichPrediction == "gaussian":
    p2 = ax[1].pcolormesh(x[::skipSome,::skipSome], y[::skipSome,::skipSome], 
                          phyStat_prediction_gaussianE0[::skipSome,::skipSome], cmap=cmap, norm=normKE)
    p3 = ax[2].pcolormesh(x[::skipSome,::skipSome], y[::skipSome,::skipSome], 
                          np.abs(KEwave_tavg[::skipSome,::skipSome] - phyStat_prediction_gaussianE0[::skipSome,::skipSome]), cmap='viridis', norm=normError)
else:
    raise ValueError("Non supported prediction")

ax[0].contour(x[::skipSome,::skipSome], y[::skipSome,::skipSome], np.sqrt(KEpsi_local[::skipSome,::skipSome]), colors="k", linewidths=0.5)
ax[1].contour(x[::skipSome,::skipSome], y[::skipSome,::skipSome], np.sqrt(KEpsi_local[::skipSome,::skipSome]), colors="k", linewidths=0.5)
ax[2].contour(x[::skipSome,::skipSome], y[::skipSome,::skipSome], np.sqrt(KEpsi_local[::skipSome,::skipSome]), colors="k", linewidths=0.5) 

fig.colorbar(p1, ax=ax[0], label="$|M|^2$")
fig.colorbar(p2, ax=ax[1], label="$|M_{pred}|^2$")
fig.colorbar(p3, ax=ax[2], label="$\\left| |M|^2 - |M_{pred}|^2 \\right|$")

ax[0].set_title(f'Kinetic energy $\\langle |M|^2 \\rangle _t$ ')
ax[1].set_title(f'Phy Stat prediction')
ax[2].set_title(f'Absolute error of the prediction')

for axi in ax:
    axi.set_xlabel('$x$')
    axi.set_ylabel('$y$')
    axi.set_aspect(1)

if whichPrediction == "full":
    savename = pathFigures + f'KE_vs_predictionForDeficit_full_{int(t_start)}_{int(t_end)}.png'
elif whichPrediction == "simple":
    savename = pathFigures + f'KE_vs_predictionForDeficit_simple_{int(t_start)}_{int(t_end)}.png'
elif whichPrediction == "gaussian":
    savename = pathFigures + f'KE_vs_predictionForDeficit_gaussian_{int(t_start)}_{int(t_end)}.png'
    

if savePlots: plt.savefig(savename, dpi=200)
plt.show()

## Compute the spatial spectrum

In [52]:
# Parameters for spectrum computation

k_max = k_xx.shape[1] / 3.  # For dealiasing 2/3 (circular otherwise missing data in corners due to dealiasing)
delta_k = 2.                # Step for radial integration

In [ ]:
# Compute the spectrum (wave and background)

bins = np.arange(0, k_max, delta_k)
spectrum = np.empty((M_hat_save.shape[0], bins.size-1))
spectrum_psi = np.empty(bins.size-1)

print(f"{bins.size-1} bins")

KEwave_hat = np.abs(M_hat_save)**2
KEpsi_hat = np.abs(psi_rms * psi_hat)**2
k_norm = np.sqrt(k_xx**2 + k_yy**2)

for i in tqdm(range(bins.size-1)):
    maskBins = (bins[i] <= k_norm) * (k_norm < bins[i+1])
    spectrum[:,i] = KEwave_hat[:,maskBins].sum(axis=1)
    spectrum_psi[i] = KEpsi_hat[maskBins].sum()

print(spectrum.shape)

In [54]:
# Time average the spectrum 
t_start = 50
t_end = np.inf

t_start = max(t_start, t.min())
t_end = min(t_end, t.max())

maskTime = (t_start <= t) & (t <= t_end)

spectrum_tavg = spectrum[maskTime].mean(axis=0)

In [ ]:
# Plot time-averaged spectrum

fig, ax = plt.subplots(figsize=(10,5), ncols=2, tight_layout=True)

for axi in ax:
    axi.plot(bins[:-1], spectrum_tavg, label="Wave $M$")
    axi.plot(bins[:-1], spectrum_psi, label="Background $\\Psi$")

    axi.set_title(f'Time-averaged power spectrum ($\\delta k = {delta_k}$)')
    axi.set_xlabel('Wavenumber $k$')
    axi.set_ylabel('Energy $E(k)$')

ax[0].set_yscale('log')
ax[0].set_xscale('linear')

ax[1].set_yscale('log')
ax[1].set_xscale('log')

ax[0].legend()
ax[0].set_ybound([1e-20, 1e4])
ax[1].set_ybound([1e-20, 1e4])

if savePlots: plt.savefig(pathFigures + f"Energy_spectrum_dk_{delta_k}_{int(t_start)}_{int(t_end)}.svg")
plt.show()